In [9]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
output_folder_csv = "../output/csv"
video_name = "test_01_my_hand"

In [7]:
df_raw_angles = pd.read_csv(os.path.join(output_folder_csv, f"{video_name}.csv"))
df_raw_angles

,frame,angle_0,angle_1,angle_2,time
0,500,108.411997,95.616618,107.868928,3.333333
1,501,108.070983,94.565037,109.502839,3.340000
2,502,108.070983,95.416016,109.118753,3.346667
3,503,108.132717,95.416016,108.271437,3.353333
4,504,109.105193,93.530827,109.122416,3.360000
...,...,...,...,...,...
583,1083,192.166393,189.383085,187.626994,7.220000
584,1084,192.166393,189.383085,186.952957,7.226667
585,1085,192.220577,187.434327,189.575753,7.233333
586,1086,192.220577,188.010393,188.325650,7.240000


# Angle / Time plot

In [355]:
# Create a Line Graph using Plotly Express
fig = px.line(df_raw_angles, x='time', y=['angle_0', 'angle_1', 'angle_2'],  labels={'value': 'Values'},title='Angles per time')
# Show the plot
fig.show()

## Smoothing demonstration

In [356]:
# Create a Line Graph using Plotly Express
window_size = 10
fig = px.line(df_raw_angles.rolling(window=window_size).mean(), x='time', y=['angle_0', 'angle_1', 'angle_2'],  labels={'value': 'Values'},title='Angles per time (smooth)')
# Show the plot
fig.show()

# Calculate angular speed
Angular speed of a rotating object: An angular speed of a rotating object is the amount of the angle changed per unit time.

I use windows averaging for smoothness

In [357]:
# Calculate delta_time and angular speeds
window_size = 10
delta_time = np.diff(df_raw_angles['time'])
angular_0_speed = np.diff(df_raw_angles['angle_0'].rolling(window=window_size).mean())
angular_1_speed = np.diff(df_raw_angles['angle_1'].rolling(window=window_size).mean())
angular_2_speed = np.diff(df_raw_angles['angle_2'].rolling(window=window_size).mean())

delta_time = np.insert(delta_time, 0, np.NAN)
angular_0_speed = np.insert(angular_0_speed, 0, np.NAN)
angular_1_speed = np.insert(angular_1_speed, 0, np.NAN)
angular_2_speed = np.insert(angular_2_speed, 0, np.NAN)


df_raw_angles["delta_time"] = delta_time
df_raw_angles["angular_0_speed"] = angular_0_speed
df_raw_angles["angular_1_speed"] = angular_1_speed
df_raw_angles["angular_2_speed"] = angular_2_speed

In [358]:
# Create a Line Graph using Plotly Express

fig = px.line(df_raw_angles.rolling(window=window_size).mean(), x='time', y=['angular_0_speed', 'angular_1_speed', 'angular_2_speed'],  labels={'value': 'Values'},title='Speed / Time (smooth)')
# Show the plot
fig.show()

## Calcualte average speed

In [151]:
window = 10
df_raw_angles['average_speed'] = np.mean(df_raw_angles[["angular_0_speed","angular_1_speed","angular_2_speed"]].rolling(window=window_size).mean(), axis=1)

In [152]:
# Create a Line Graph using Plotly Express
window = 10
fig = px.line(df_raw_angles.rolling(window=window_size).mean(), x='time', y=['angular_0_speed', 'angular_1_speed', 'angular_2_speed',],  labels={'value': 'Values'},title='Speed / Time (smooth)')
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'].rolling(window=window).mean(),
                         mode='lines', name=f'Average Speed', line=dict(color='black')))
# Show the plot
fig.show()

# Is finger moving?

In [338]:
threshold = 0.2
# df_raw_angles["is_move"] = (np.abs(df_raw_angles['average_speed'].rolling(window=window).mean())  > threshold) | (np.abs(df_raw_angles['angular_0_speed'].rolling(window=window).mean())  > threshold)
# df_raw_angles["is_move"] = np.abs(df_raw_angles['average_speed'].rolling(window=window).mean())  > threshold
window = 10
angle_columns = ["angle_0", "angle_1", "angle_2"]
speed_columns = ['angular_0_speed', 'angular_1_speed', 'angular_2_speed']
# df_raw_angles["is_move"] = np.any(np.abs(df_raw_angles[['angular_0_speed', 'angular_1_speed', 'angular_2_speed']].rolling(window=window).mean())  > threshold,axis=1)

df_raw_angles["is_move"] = np.abs(df_raw_angles["average_speed"]) > threshold

In [359]:
# Create a Line Graph using Plotly Express
window = 10
fig = px.line(df_raw_angles, x='time', y=['angular_0_speed', 'angular_1_speed', 'angular_2_speed'],  labels={'value': 'Values'},title='Speed / Time (smooth)')
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'].max() * df_raw_angles["is_move"],
                         fill='toself', fillcolor='rgba(255, 0, 0, 0.2)',
                         line=dict(color='rgba(255, 255, 255, 0)'),
                         name=f'Is move', showlegend=False))
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'].max() * df_raw_angles["is_move"] * -1,
                         fill='toself', fillcolor='rgba(255, 0, 0, 0.2)',
                         line=dict(color='rgba(255, 255, 255, 0)'),
                         name=f'Is move', showlegend=False))

fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'],
                         mode='lines', name=f'Average Speed', line=dict(color='black')))
# Show the plot
fig.show()

In [340]:
# Create a Line Graph using Plotly Express
window = 10
fig = px.line(df_raw_angles, x='time', y=['angle_0', 'angle_1', 'angle_2'],  labels={'value': 'Values'},title='Speed / Time (smooth)')
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'].max() * df_raw_angles["is_move"] * 40 ,
                         fill='toself', fillcolor='rgba(255, 0, 0, 0.2)',
                         line=dict(color='rgba(255, 255, 255, 0)'),
                         name=f'Is move', showlegend=False))

# fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'].rolling(window=window).mean(),
#                          mode='lines', name=f'Average Speed', line=dict(color='black')))
# Show the plot
fig.show()

# Which way finger moving?

In [360]:
# Create masks for positive and negative 'move' regions
# df_raw_angles["positive_move_mask"] = (df_raw_angles["is_move"]) & (df_raw_angles["average_speed"] > 0.0)
# df_raw_angles["negative_move_mask"] = (df_raw_angles["is_move"]) & (df_raw_angles["average_speed"] < -0.0)

window= 10

# df_raw_angles["positive_move_mask"] = (df_raw_angles["is_move"]) & (np.all(df_raw_angles[speed_columns].rolling(window=window).mean() > 0.0, axis=1))
# df_raw_angles["negative_move_mask"] = (df_raw_angles["is_move"]) & (np.all(df_raw_angles[speed_columns].rolling(window=window).mean() < 0.0, axis=1))

df_raw_angles["positive_move_mask"] = (df_raw_angles["is_move"]) & np.all(
    np.array([df_raw_angles["angular_0_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) > 0,
              df_raw_angles["angular_1_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) > 0,
              df_raw_angles["angular_2_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) > 0] ),axis=0)
# df_raw_angles["negative_move_mask"] = (df_raw_angles["is_move"]) & (np.all(df_raw_angles[speed_columns].rolling(window=window).mean() < 0.0, axis=1))
df_raw_angles["negative_move_mask"] = (df_raw_angles["is_move"]) & np.all(
    np.array([df_raw_angles["angular_0_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) < 0,
              df_raw_angles["angular_1_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) < 0,
              df_raw_angles["angular_2_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) < 0] ),axis=0)


In [361]:
# Create a Line Graph using Plotly Express
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, shared_yaxes=True)

window= 10
# # Calculate the average speed of the three angles
# df_angle_speed["average_speed"] = df_raw_angles[speed_columns].mean(axis=1)

for speed_column in speed_columns:
    # Plot smoothed angle data
    fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles[speed_column].rolling(window=window).mean(),
                             mode='lines', name=f'{speed_column}'))

# Add shaded regions for positive 'move' intervals
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=np.mean(df_raw_angles[speed_columns],axis=1).max() * df_raw_angles["positive_move_mask"],
                         fill='toself', fillcolor='rgba(255, 0, 0, 0.2)',
                         line=dict(color='rgba(255, 255, 255, 0)'),
                         name=f'Positive Move Region - {speed_columns}', showlegend=False))

# Add shaded regions for negative 'move' intervals
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=np.mean(df_raw_angles[speed_columns],axis=1).max()* df_raw_angles["negative_move_mask"] * -1,
                         fill='toself', fillcolor='rgba(0, 0, 255, 0.2)',
                         line=dict(color='rgba(255, 255, 255, 0)'),
                         name=f'Negative Move Region - {speed_columns}', showlegend=False))

# Plot the average speed
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'],
                         mode='lines', name=f'Average Speed', line=dict(color='black')))

# Update layout
fig.update_layout(title_text=f'Angles per frame with Move Regions and Average Speed')

# Show the plot
fig.show()


In [343]:
df_raw_angles["positive_move_mask"]

0      False
1      False
2      False
3      False
4      False
       ...  
583    False
584    False
585    False
586    False
587    False
Name: positive_move_mask, Length: 588, dtype: bool

In [344]:
df_raw_angles["positive_move_mask"].sum()

157

In [345]:
np.all(
    np.array([df_raw_angles["angle_0"].rolling(window,center = True).apply(lambda x: sum(sorted(x)[1:window-1])) > 0,
     df_raw_angles["angle_1"].rolling(window,center = True).apply(lambda x: sum(sorted(x)[1:window-1])) > 0,
     df_raw_angles["angle_1"].rolling(window,center = True).apply(lambda x: sum(sorted(x)[1:window-1])) > 0] ),axis=0)

array([False, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [325]:
df_raw_angles["angle_0"].rolling(10).apply(lambda x: np.mean(sorted(x)[1:10-1]))

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
583    192.336686
584    192.289305
585    192.296078
586    192.232573
587    192.209609
Name: angle_0, Length: 588, dtype: float64

In [324]:
df_raw_angles["angle_0"]

0      108.411997
1      108.070983
2      108.070983
3      108.132717
4      109.105193
          ...    
583    192.166393
584    192.166393
585    192.220577
586    192.220577
587    191.260977
Name: angle_0, Length: 588, dtype: float64